In [1]:
!git clone https://github.com/OpenBMB/CPM-Bee.git

Cloning into 'CPM-Bee'...
remote: Enumerating objects: 233, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 233 (delta 5), reused 16 (delta 4), pack-reused 201
Receiving objects: 100% (233/233), 545.93 KiB | 12.70 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [2]:
%cd CPM-Bee/src

/content/CPM-Bee/src


In [3]:
!git clone https://huggingface.co/openbmb/cpm-bee-1b

Cloning into 'cpm-bee-1b'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), 312.41 KiB | 1.27 MiB/s, done.


In [4]:
!pip install bmtrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bmtrain: filename=bmtrain-0.2.2-cp310-cp310-linux_x86_64.whl size=13412034 sha256=339ed79ea3d1ace40c042546505b826fcac89998cf66bc2e8ba4a4f89877b872
  Stored in directory: /root/.cache/pip/wheels/2c/21/2d/a9b120b051f06f18bdca81e4862210b83795032deec040aa5c
Successfully built bmtrain


In [5]:
from cpm_live.generation.bee import CPMBeeBeamSearch
from cpm_live.models import CPMBeeTorch, CPMBeeConfig
from cpm_live.tokenizers import CPMBeeTokenizer
import torch


config = CPMBeeConfig.from_json_file("cpm-bee-1b/config.json")
ckpt_path = "cpm-bee-1b/pytorch_model.bin"
tokenizer = CPMBeeTokenizer()
model = CPMBeeTorch(config=config)

model.load_state_dict(torch.load(ckpt_path))
model.cuda()

beam_search = CPMBeeBeamSearch(
    model=model,
    tokenizer=tokenizer,
)

{'document': '今天天气是真的<mask_0>', '<ans>': {'<mask_0>': '好'}}


In [8]:
# 填空
data_list = [
{"input": """心理学领域的研究人员发现，做出重要决定的最好方法之一，比如选择一所大学或<mask_0>，
       都涉及到使用决策工作表。研究优化的心理学家将<mask_1>与理论理想决策进行比较，看看它们有多相似。
       工作表程序的支持者认为它会产生最优的，也就是说，最好的决策。虽然有<mask_2>可以接受，但它们在本质上都是相似的。""",
 "<ans>":{"<mask_0>":"","<mask_1>":"","<mask_2>":""}}
]

inference_results = beam_search.generate(data_list, max_length=100, repetition_penalty=1.1)

inference_results[0]["<ans>"]

{'<mask_0>': '者一个专业', '<mask_1>': '决策工作表', '<mask_2>': '些人认为它不'}

In [9]:
# 文本生成
data_list = [
{"input": "今天天气很好，我和妈妈一起去公园，", 
 "prompt": "往后写约100字", 
 "<ans>": ""}
]

inference_results = beam_search.generate(data_list, max_length=100, repetition_penalty=1.1)

inference_results[0]["<ans>"]

'公园里有很多人，我和妈妈走在最前面。\n我们走到一半的时候，突然一个人从后面冲了过来，把我撞倒在地。\n“你没事吧？”我问他。\n“没事啊！”他说。\n“那就好。”我说。\n“你怎么会来这里呢？”他问。\n“我来给你们送吃的呀！”我说。\n“哦！”他说。'

In [10]:
# 翻译
data_list = [
    {"input": "北京是中国的首都", 
     "prompt": "中翻英", 
     "<ans>": ""}
]

inference_results = beam_search.generate(data_list, max_length=100, repetition_penalty=1.1)

inference_results[0]["<ans>"]

'Beijing is the capital of China'

In [11]:
# 问答
data_list = [
{"input": """NGC 6231是一个位于天蝎座的疏散星团，天球座标为赤经16时54分，
       赤纬-41度48分，视觉观测大小约45角分，亮度约2.6视星等，距地球5900光年。
       NGC 6231年龄约为三百二十万年，是一个非常年轻的星团，星团内的最亮星是5等的天蝎座 ζ1星。
       用双筒望远镜或小型望远镜就能看到个别的行星。
       NGC 6231在1654年被意大利天文学家乔瓦尼·巴蒂斯特·霍迪尔纳（Giovanni Battista Hodierna）以Luminosae的名字首次纪录在星表中，
       但是未见记载于夏尔·梅西耶的天体列表和威廉·赫歇尔的深空天体目录。
       这个天体在1678年被爱德蒙·哈雷（I.7）、1745年被夏西亚科斯（Jean-Phillippe Loys de Cheseaux）（9）、
       1751年被尼可拉·路易·拉卡伊（II.13）分别再次独立发现。""", 
 "question": "NGC 6231的经纬度是多少？", 
 "<ans>": ""}
]

inference_results = beam_search.generate(data_list, max_length=100, repetition_penalty=1.1)

inference_results[0]["<ans>"]

'赤纬-41度48分'

In [12]:
# 评分预测
data_list = [
{"input":"""之前多次聚餐都选择这里，有各种大小的包房同时能容纳很多人，
      环境好有特色还有表演，整体聚餐氛围一下被带动起来。
      现在由于炭火改成了电烤羊，口感真的不如从前，不过其他菜品都还是不错，
      烤羊剩下的拆骨肉最后还能再加工一下椒盐的也很好吃。""",
 "question":"评分是多少？(1-5)",
 "<ans>":""},

]

inference_results = beam_search.generate(data_list, max_length=100, repetition_penalty=1.1)

inference_results[0]["<ans>"]

'4分'

In [13]:
# 选择题
data_list = [
{"input": """父母都希望自己的孩子诚实、勇敢、有礼貌。要想让孩子成为这样的人，
       父母首先得从自己做起，要是连自己都做不到，又怎能要求孩子做到呢？""", 
 "options": {"<option_0>": "少提要求", "<option_1>": "降低标准", 
        "<option_2>": "自己先做好", "<option_3>": "让孩子拿主意"}, 
 "question": "教育孩子时，父母应该：", 
 "<ans>": ""}
]

inference_results = beam_search.generate(data_list, max_length=100, repetition_penalty=1.1)

inference_results[0]["<ans>"]

'<option_2>'